### Tensor Flow 2-11 notebook

Notes: 
- Notebook should be running with an Nvidia GPU to for top performance
- load_img will load a nparray with x, y, 3 color dims.  
- If we're just processing one image we will need to np.expand_dims(image, axies=0) to get 1, x, y, 3. The library expects N images in the first dim
- should set random seed on tf.random.set_seed(1) for reproduceability 
- 

last change 3/6/2025

In [1]:
# import the libraries for training, testing, validation
import tensorflow as tf
from tensorflow.keras.applications import imagenet_utils  # will decode predictions out of the model into a 4 dim array of N (image num), imageID, label, probability result[0] would be the set of results for image one
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img  # will load img and reshape, usage is load_img(image_name_loc, target_size=input_shape)
from tensorflow.keras.utils import plot_model  # Note: usage syntax is plot_model(instantied_model, to_file='', show_shapes=True)

import matplotlib.pyplot as plt
import numpy as np

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
import tensorflow_datasets as tfds  # For loading datasets from GCS
import os


2025-03-09 14:18:38.119947: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-03-09 14:18:40.668052: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/local/cuda/extras/CUPTI/lib64:/usr/lib/x86_64-linux-gnu/:/opt/conda/lib
2025-03-09 14:18:40.668221: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local

In [2]:
# training packages
# import the necessary packages
# from tensorflow import reduce_mean
# from tensorflow.keras import Model
# from tensorflow.keras import Sequential
# from tensorflow.keras.layers import Conv2D
# from tensorflow.keras.layers import InputLayer
# from tensorflow.keras.layers import Conv2DTranspose
# from tensorflow.keras.losses import Reduction
# from tensorflow.keras.losses import MeanSquaredError
# from tensorflow.image import convert_image_dtype
# from tensorflow.keras.preprocessing.image import array_to_img

# import tensorflow_datasets as tfds

In [3]:
# Configuration
GCS_BUCKET = 'nabirds_filtered'  
DATASET_PATH = 'images'  # Relative path within the bucket
IMAGE_SIZE = (224, 224)
BATCH_SIZE = 32
NUM_CLASSES = None  # Will be determined from the dataset

In [4]:
def load_gcs_dataset(bucket_name, dataset_path, image_size, batch_size):
    """Loads a dataset from Google Cloud Storage."""

    gcs_dataset_path = f"gs://{bucket_name}/{dataset_path}"

    try:
        # Attempt to load the dataset using tf.keras.utils.image_dataset_from_directory.
        # This handles directory structures well.

        dataset = keras.utils.image_dataset_from_directory(
            gcs_dataset_path,
            image_size=image_size,
            batch_size=batch_size,
            label_mode='categorical', # important for softmax
        )
        return dataset
    except Exception as e:
        print(f"Error loading dataset from GCS: {e}")
        return None


In [5]:
train_dataset = load_gcs_dataset(GCS_BUCKET, os.path.join(DATASET_PATH, 'train'), IMAGE_SIZE, BATCH_SIZE)
validation_dataset = load_gcs_dataset(GCS_BUCKET, os.path.join(DATASET_PATH, 'test'), IMAGE_SIZE, BATCH_SIZE)

if train_dataset is None or validation_dataset is None:
    print("Dataset loading failed. Exiting.")
    exit()

NUM_CLASSES = len(train_dataset.class_names) # Get class count.


Found 2455 files belonging to 27 classes.


2025-03-09 14:26:45.519112: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/local/cuda/extras/CUPTI/lib64:/usr/lib/x86_64-linux-gnu/:/opt/conda/lib
2025-03-09 14:26:45.519143: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2025-03-09 14:26:45.519172: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (bird-feeder-profiling-2024-2025): /proc/driver/nvidia/version does not exist
2025-03-09 14:26:45.522482: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To ena

Found 621 files belonging to 27 classes.


In [6]:

# --- 2. Load MobileNetV2 and Modify Head ---
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Unlock the base model (you can choose to freeze some layers if needed)
base_model.trainable = True

# Add custom layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)  # Optional: Add more dense layers
predictions = Dense(NUM_CLASSES, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

# --- 3. Compile the Model ---
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0001),
              loss='categorical_crossentropy',  # Important for multi-class classification
              metrics=['accuracy'])

# --- 4. Train the Model ---
epochs = 10  # Adjust as needed

history = model.fit(
    train_dataset,
    epochs=epochs,
    validation_data=validation_dataset
)

# --- 5. Save the Model (Optional) ---
model.save('mobilenet_retrained.h5')

print("Training complete.")

9406464/9406464 [==============================] - 0s 0us/step
Epoch 1/10


2025-03-09 14:28:27.768350: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:392] Filling up shuffle buffer (this may take a while): 92 of 256
2025-03-09 14:28:37.855424: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:392] Filling up shuffle buffer (this may take a while): 187 of 256
2025-03-09 14:28:45.256797: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:417] Shuffle buffer filled.


77/77 [==============================] - ETA: 0s - loss: 1.9390 - accuracy: 0.4908

2025-03-09 14:33:40.395498: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:392] Filling up shuffle buffer (this may take a while): 92 of 256
2025-03-09 14:33:50.414424: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:392] Filling up shuffle buffer (this may take a while): 189 of 256
2025-03-09 14:33:57.230597: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:417] Shuffle buffer filled.


77/77 [==============================] - 391s 5s/step - loss: 1.9390 - accuracy: 0.4908 - val_loss: 2.7848 - val_accuracy: 0.1965
Epoch 2/10


...

77/77 [==============================] - 345s 4s/step - loss: 0.0119 - accuracy: 0.9992 - val_loss: 0.6622 - val_accuracy: 0.7971
Epoch 10/10


2025-03-09 15:30:31.924979: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:392] Filling up shuffle buffer (this may take a while): 116 of 256
2025-03-09 15:30:41.857610: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:392] Filling up shuffle buffer (this may take a while): 231 of 256
2025-03-09 15:30:44.146693: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:417] Shuffle buffer filled.


77/77 [==============================] - ETA: 0s - loss: 0.0078 - accuracy: 0.9996

2025-03-09 15:35:21.684198: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:392] Filling up shuffle buffer (this may take a while): 114 of 256
2025-03-09 15:35:31.718052: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:392] Filling up shuffle buffer (this may take a while): 236 of 256
2025-03-09 15:35:33.325738: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:417] Shuffle buffer filled.


77/77 [==============================] - 348s 4s/step - loss: 0.0078 - accuracy: 0.9996 - val_loss: 0.6094 - val_accuracy: 0.8068
Training complete.
